In [1]:
import pandas as pd
import glob
import numpy as np

In [2]:
#set up path to all the files
path = '../Data/Raw'
all_files = glob.glob(path + '/*.csv')

In [3]:
list_df = []
for file in all_files:
    temp_df = pd.read_csv(file)
    list_df.append(temp_df)
df = pd.concat(list_df)

C:\Users\sadmin\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [4]:
df.head()

,Confirmed,Country,Country/Region,Date last updated,Death,Deaths,Last Update,Province/State,Recovered,Suspected,Unnamed: 0
0,9.0,Mainland China,NaN,1/21/2020,NaN,0.0,NaN,Shanghai,0.0,10.0,0.0
1,1.0,Mainland China,NaN,1/21/2020,NaN,0.0,NaN,Yunnan,0.0,0.0,1.0
2,10.0,Mainland China,NaN,1/21/2020,NaN,0.0,NaN,Beijing,0.0,0.0,2.0
3,1.0,Taiwan,NaN,1/21/2020,NaN,0.0,NaN,Taiwan,0.0,0.0,3.0
4,0.0,Mainland China,NaN,1/21/2020,NaN,0.0,NaN,Jilin,0.0,1.0,4.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8868 entries, 0 to 1876
Data columns (total 11 columns):
Confirmed            8659 non-null float64
Country              368 non-null object
Country/Region       8500 non-null object
Date last updated    368 non-null object
Death                2386 non-null float64
Deaths               368 non-null float64
Last Update          8500 non-null object
Province/State       6971 non-null object
Recovered            3410 non-null float64
Suspected            984 non-null float64
Unnamed: 0           368 non-null float64
dtypes: float64(6), object(5)
memory usage: 831.4+ KB


In [6]:
#fill null values of country with Country/Region
df['Country'] = df['Country'].fillna('')
df['Country'] = df.apply(lambda row: row['Country/Region'] if not row.Country else row.Country, axis = 1)

#fill null values of last update with date last updated
df['Last Update'] = df['Last Update'].fillna('')
df['Last Update'] = df.apply(lambda row: row['Date last updated'] if not row['Last Update'] else row['Last Update'], axis = 1)

#filled null of death with 0 or Deaths Feature value
df['Death'] = df['Death'].fillna(0)
df['Death'] = df.apply(lambda row: row['Deaths'] if row.Deaths > 0 else row.Death, axis = 1)

#filled null values of Confirmed, Recovered, and Suspected with 0
df['Confirmed'] = df['Confirmed'].fillna(0)
df['Recovered'] = df['Recovered'].fillna(0)
df['Suspected'] = df['Suspected'].fillna(0)

#drop the unused columns
df = df.drop(columns = ['Date last updated', 'Country/Region', 'Deaths', 'Unnamed: 0'])

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8868 entries, 0 to 1876
Data columns (total 7 columns):
Confirmed         8868 non-null float64
Country           8868 non-null object
Death             8868 non-null float64
Last Update       8868 non-null object
Province/State    6971 non-null object
Recovered         8868 non-null float64
Suspected         8868 non-null float64
dtypes: float64(4), object(3)
memory usage: 554.2+ KB


In [7]:
#change last update to datetime
df['Last Update'] = pd.to_datetime(df['Last Update'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8868 entries, 0 to 1876
Data columns (total 7 columns):
Confirmed         8868 non-null float64
Country           8868 non-null object
Death             8868 non-null float64
Last Update       8868 non-null datetime64[ns]
Province/State    6971 non-null object
Recovered         8868 non-null float64
Suspected         8868 non-null float64
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 554.2+ KB


In [8]:
df.to_csv('../Data/Processed/Processed.csv')